In [108]:
import pandas as pd
import urllib.parse
from random import shuffle
from my_rymscraper.rymscraper import rymscraper, RymUrl
network = rymscraper.RymNetwork()

In [109]:
mt = pd.read_csv("metacritic_data.csv")

In [110]:
# mt['tag'] = mt["Singer"] + " - " + mt["Album_Name"];
lower_alnum = lambda s: "".join(x for x in s.lower() if x.isalnum())
mt["id"] = mt["Singer"].apply(lower_alnum) + mt["Release_Year"].apply(str) + mt["Album_Name"].apply(lambda s: lower_alnum(s)[:3] + lower_alnum(s)[-3:])
tags = list(zip(mt["id"], mt["Singer"], mt["Album_Name"]))
# tags = ["Common - A Beautiful Revolution, Vol. 2"]
cols = ['Id', 'Name', 'Artist', 'Released', 'RYM Rating', 'Genres', 'Descriptors',
       'Language', 'Track listing', 'Colorscheme', 'Languages', 'Ranked']
df = pd.DataFrame(columns=cols)

In [112]:
lower_alnum_dash = lambda s: "".join(x for x in s.replace(" ", "-").lower() if (x.isalnum() or x == "-" ))
# tags = [('divmo','Maya Hawke', 'MOSS')]

max_iter = 1 # one iteration is enough for most cases, raise it if errors occur
iter = 0
while iter < max_iter:
  new_tags = []
  for uid, singer, album in tags:
    try:
      # Try url first
      url = "https://rateyourmusic.com/release/album/%s/%s/" % (lower_alnum_dash(singer), lower_alnum_dash(album))
      tag = singer + " - " + album
      album_infos = network.get_album_infos(url=url)
    except:
      album_infos = []
    
    if not album_infos:
      try:
        # Try search then
        url_tag = urllib.parse.quote(tag)
        album_infos = network.get_album_infos(name=url_tag) # get album by search
      except IndexError as e:
        print("❌ ", tag)
      
    if not album_infos:
      new_tags.append((uid, singer, album))
    else:
      print("✅ ", tag)
      album_infos["Id"] = uid
      df = pd.concat([df, pd.DataFrame([album_infos])], axis=0)
    # except IndexError:
    # except (IndexError, AttributeError):

  tags = new_tags
  iter += 1

✅  Go - Kart Mozart
❌  Steve Reich - Pulse/Quartet
✅  µ - Ziq
❌  Christina Vantzou - No. 4
❌  Coheed and Cambria - Vaxis
❌  Beak> - 3 [>>>]
❌  Snoh Aalegra - 
❌  Gil Scott - Heron
❌  Daniel Blumberg - On&On
❌  Sharon Jones & the Dap - Kings
❌  Metallica - S&M2 [Live]
❌  Lukas Nelson & Promise of the Real - A Few Stars Apart
❌  Ed Sheeran - Equals
❌  Bill Callahan - YTILAER
❌  ...And You Will Know Us the Trail of Dead - XI: Bleed Here Now
❌  Sonic Youth - In/Out/In
❌  Marcus Mumford - (self


Rate-limit detected. Restarting browser.


❌  St. Paul & the Broken Bones - The Alien Coast


In [123]:
new_tags = [('stevereich2018pultet', 'Steve Reich', 'Pulse/Quartet'),
 ('christinavantzou2018no4no4', 'Christina Vantzou', 'No. 4'),
 ('coheedandcambria2018vaxres', 'Coheed and Cambria', 'Vaxis'),
 ('beak201833', 'Beak>', '3 [>>>]'),
 ('snohaalegra2019ughain', 'Snoh Aalegra', ''),
 ('gilscottheron2020werven', 'Gil Scott', 'Heron'),
 ('danielblumberg2020ononon', 'Daniel Blumberg', 'On&On'),
 ('sharonjonesthedapkings2020jussin', 'Sharon Jones & the Dap', 'Kings'),
 ('metallica2020sm2ive', 'Metallica', 'S&M2 [Live]'),
 ('lukasnelsonpromiseofthereal2021afeart',
  'Lukas Nelson & Promise of the Real',
  'A Few Stars Apart'),
 ('edsheeran2021equals', 'Ed Sheeran', 'Equals'),
 ('billcallahan2022ytiaer', 'Bill Callahan', 'YTILAER'),
 ('andyouwillknowusthetrailofdead2022xibnow',
  '...And You Will Know Us the Trail of Dead',
  'XI: Bleed Here Now'),
 ('sonicyouth2022inotin', 'Sonic Youth', 'In/Out/In'),
 ('marcusmumford2022selled', 'Marcus Mumford', '(self'),
 ('stpaulthebrokenbones2022theast', 'St. Paul & the Broken Bones', 'The Alien Coast')]

In [113]:
xdf = df.reset_index()
xdf = df[cols]
xdf.reset_index()
xdf.describe()

,Id,Name,Artist,Released,RYM Rating,Genres,Descriptors,Language,Track listing,Colorscheme,Languages,Ranked
count,2,2,2,2,1,2,2,0,2,2,0,1
unique,2,2,2,2,1,2,1,0,2,2,0,1
top,gokartmozart2018mozart,Mozart,Sky,1987,3.18\n \n/ 5.00.5\n\n ...,"Classical Crossover, Symphonic Prog",,NaN,"[The Marriage of Figaro: Overture, Eine Kleine...","[#a03b11, #b49986, #8b8b7f, #b37743]",NaN,"#1,797 for 1987"
freq,1,1,1,1,1,1,2,NaN,1,1,NaN,1


In [114]:
xdf["Language"] = [xdf.iloc[i]["Languages"] if pd.isna(xdf.iloc[i]["Language"]) else xdf.iloc[i]["Language"] for i in range(len(xdf.index))]

/opt/homebrew/anaconda3/envs/edav/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [115]:
xdf = xdf[['Id', 'Name', 'Artist', 'Released', 'RYM Rating', 'Genres', 'Descriptors',
       'Language', 'Track listing', 'Colorscheme','Ranked']]
xdf.describe()

,Language
count,0.0
mean,NaN
std,NaN
min,NaN
25%,NaN
50%,NaN
75%,NaN
max,NaN


In [116]:
import re
ratings = []
num_ratings = []
for i in xdf["RYM Rating"]:
  if isinstance(i, float):
    ratings.append("")
    num_ratings.append("")
  else:
    ratings.append(float(i.rsplit('\n')[0]))
    num_ = i.rsplit('\n')[-1]
    num = int(re.findall(r"\d*,*\d+",num_)[0].replace(",",""))
    num_ratings.append(num)

In [117]:
xdf["RYM Rating"] = ratings
xdf["Num_of_RYM_Ratings"] = num_ratings

In [118]:
xdf

,Id,Name,Artist,Released,RYM Rating,Genres,Descriptors,Language,Track listing,Colorscheme,Ranked,Num_of_RYM_Ratings
0,gokartmozart2018mozart,Mozart,Sky,1987,3.18,"Classical Crossover, Symphonic Prog",,NaN,"[The Marriage of Figaro: Overture, Eine Kleine...","[#a03b11, #b49986, #8b8b7f, #b37743]","#1,797 for 1987",36
0,µziq2018chaish,Ziq,Conga Squad,1998,,Techno,,NaN,"[Ziq, Ziq (Original), Droids, Tell me something]",[],NaN,


In [119]:
xdf.to_csv("rym_add.csv")